In [121]:
%load_ext autoreload
%autoreload 2
import discrimination
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Activation, Flatten, Dropout
from keras.models import Sequential
from keras import regularizers
import itertools
import pickle
import random
import re
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
### Split sexist texts into sentences and tokenize
---

In [122]:
# Load texts and only keep texts (from tuples loaded)
sex_txts = pickle.load(open("pickles2/sexist_new.p", "rb"))
sex_txts = [txt[0] for txt in sex_txts]
# Split into a list of sentences
sex_temp = discrimination.texts.sentences_split(sex_txts)
# Combine all lists in one big list of sentences
sex_sentences = []
for item in sex_temp:
    for sentence in item:
        sex_sentences.append(sentence)
# Uncensore bad words!
for j in range(5):
    for i, item in enumerate(sex_sentences):
        if "c~~~" in item:
            sex_sentences[i] =  item.replace("c~~~", "cunt")
        if "s~~~" in item:
            sex_sentences[i] =  item.replace("s~~~", "shit")
        if "f~~~" in item:
            sex_sentences[i] =  item.replace("f~~~", "fuck")
        if "p~~~" in item:
            sex_sentences[i] =  item.replace("p~~~", "piss")
        if "t~~~" in item:
            sex_sentences[i] =  item.replace("t~~~", "tits")
        if "b~~~~" in item:
            sex_sentences[i] =  item.replace("b~~~~", "balls")
        if "fuck~~" in item:
            sex_sentences[i] =  item.replace("fuck~~", "fucker")
        if "f~~" in item:
            sex_sentences[i] =  item.replace("f~~", "fag")       
for j in range(5):
    for i, item in enumerate(sex_sentences):
        if "c***" in item:
            sex_sentences[i] =  item.replace("c***", "cunt")
        if "s***" in item:
            sex_sentences[i] =  item.replace("s***", "shit")
        if "f***" in item:
            sex_sentences[i] =  item.replace("f***", "fuck")
        if "p***" in item:
            sex_sentences[i] =  item.replace("p***", "piss")
        if "t***" in item:
            sex_sentences[i] =  item.replace("t***", "tits")
        if "b****" in item:
            sex_sentences[i] =  item.replace("b****", "balls")
        if "fuck**" in item:
            sex_sentences[i] =  item.replace("fuck**", "fucker")
        if "f****" in item:
            sex_sentences[i] =  item.replace("f****", "fucking")        
        if "a**" in item:
            sex_sentences[i] =  item.replace("a**", "ass")  
        if "s**t" in item:
            sex_sentences[i] =  item.replace("s**t", "shit")   

In [123]:
# Tokenize sentences and remove stop-words
sex_tokens = discrimination.texts.tokenize(sex_sentences)
# Spell-check tokens. This actually takes some time (not too much) so there's a timer every 20.000 tokens checked.
sex_tokens = discrimination.texts.spellcheck_tokens(sex_tokens)
# Remove stop-words a second time, in case some stopwords where misspelled.
sex_tokens = discrimination.texts.remove_stopwords(sex_tokens)
# Remove empty tokens and the relevant sentences.
temp_tokens = sex_tokens.copy()
temp_sentences = sex_sentences.copy()
sex_tokens.clear()
sex_sentences.clear()
for i, tkn in enumerate(temp_tokens):
    if len(tkn) != 0:
        sex_tokens.append(tkn)
        sex_sentences.append(temp_sentences[i])

100000 tokens spell-checked.
200000 tokens spell-checked.
300000 tokens spell-checked.


In [124]:
print(len(sex_tokens),"sexist sentences.")

350792 sexist sentences.


In [125]:
# Save
pickle.dump(sex_tokens, open("pickles5/sex_tokens.p", "wb"))
pickle.dump(sex_sentences, open("pickles5/sex_sentences.p", "wb"))

In [128]:
random.sample(sex_sentences, 10)

[" i mean i understand people against misandry but what's wrong with equality?",
 " and i'm not a feminist because this title promotes female 'equality' but they never talk about men equality.",
 'this is the most sexist thing i\'ve seen i don\'t apply to any of those things this video is offensive if you think you\'re "feminist" and want equality, then go be a trash man or build something with you\'re own two hands, go do something that women don\'t traditionally do, then say you\'re a feminist',
 ' so to hide their grandiose expectations, they paint half of men as slobs who are not even worthy of causal sex!  /n if you are an ugly, confused man.',
 '"\xa0it is certainly the case that some feminists are sometimes annoying, over-dramatic, wrong, etc.',
 'how does this situation have anything to do with what we are discussing?',
 '  /n however, i do not, as again, blame women, or feminists.',
 ' who the fuck veers their truck off the road just to yell expletives at a fat bearded man?',
